In [ ]:
#import dependencies
from random import choice
import random
import heapq
import pandas as pd
import numpy as np
import math
from deap import base, creator, tools, algorithms
from plot_utils import *
from collections import defaultdict, deque, Counter
import time

#read data
position_data = pd.read_csv("sub_data_file_with_header.csv")
#position_data.head()

# #add the base station to the dataframe if not added
# new_rows = pd.DataFrame({'Nos.': [len(position_data) + 1, len(position_data) + 2],
#                          'x': [5000, -5000],
#                          'y': [-5000, 5000]})
# position_data = pd.concat([position_data, new_rows], ignore_index=True)

#extract the index, x, y
i = position_data.loc[:, 'No.']
x = position_data.loc[:,'x']
y = position_data.loc[:,'y']

#define function for distance and transmission rate

def distance(x1, x2, y1, y2):
    return math.sqrt(((x2 - x1) ** 2) + ((y2 - y1) ** 2))

def transmission_rate(d):
    if d >= 3000:
        return 0
    elif d >= 2500 and d <3000:
        return 1
    elif d >= 2000 and d < 2500:
        return 2
    elif d>= 1500 and d < 2000:
        return 3
    elif d >= 1000 and d < 1500:
        return 4
    elif d >= 500 and d < 1000:
        return 5
    elif d >0 and d< 500:
        return 7
    else:
        return 0

#generate the graph using the predefined functions
graph = {}
for r,x_1,y_1 in zip(i,x,y):
    graph["Node_" + str(r)] = {}
    for j, x_2,y_2 in zip(i,x,y):
        #if statement to chech if x1,y1,x2,y2 are the same, and continue
        if (x_1 == x_2) and (y_1 == y_2):
            continue
        #call the distance function, returns transmission rate
        dist = distance(x_1, x_2, y_1, y_2)
        tr_rate = transmission_rate(dist)
        #check if the transmission is greater than zero
        if tr_rate > 0:
            graph["Node_" + str(r)]["Node_" + str(j)] = tr_rate


#Helper functions
#Helps get transmission rate in a path
def get_tran_rate(bp):
    rates = []
    for i in range(len(bp)-1):
        rates.append(graph[bp[i]][bp[i+1]])
    return min(rates)

#Helps get row number to make plots easier
def extract_point(bp):
    all_value = []
    for i in bp:
        all_value.append(int(i.split("_")[1]) -1)
    return all_value

In [ ]:
import random
from random import choice

# Hyperparameters
max_trans_rate = 7

# --------------
# Pareto Helpers
# --------------
def dominates(cost_a, cost_b):
    """
    cost_a = (rate_a, lat_a)
    cost_b = (rate_b, lat_b)
    We want to MAXIMIZE 'rate' and MINIMIZE 'latency'.
    cost_a dominates cost_b if:
      rate_a >= rate_b AND latency_a <= latency_b
      and they are not exactly the same in both.
    """
    rate_a, lat_a = cost_a
    rate_b, lat_b = cost_b

    return (
        (rate_a >= rate_b) and
        (lat_a <= lat_b) and
        (cost_a != cost_b)
    )

def non_dominated_sort(costs):
    """
    costs is a list of (rate, latency) pairs for each individual.
    Returns a list of 'fronts', where each front is a list of indices
    into costs. front[0] is the set of indices for the first (best) front,
    front[1] is the second, etc.
    """
    population_size = len(costs)

    # For each solution, store:
    #   S[i] = set of solutions that i dominates
    #   n[i] = how many solutions dominate i
    S = [set() for _ in range(population_size)]
    n = [0] * population_size

    for i in range(population_size):
        for j in range(population_size):
            if i == j:
                continue
            if dominates(costs[i], costs[j]):
                # i dominates j
                S[i].add(j)
            elif dominates(costs[j], costs[i]):
                # j dominates i
                n[i] += 1

    # Now iteratively build the fronts
    fronts = [[]]
    # first front: all i with n[i] == 0
    fronts[0] = [i for i in range(population_size) if n[i] == 0]

    f = 0
    # Go through each front
    while True:
        Q = []
        # For each solution p in front f
        for p in fronts[f]:
            # For each solution q dominated by p
            for q in S[p]:
                n[q] -= 1
                if n[q] == 0:
                    Q.append(q)
        if not Q:
            break
        # Next front
        fronts.append(Q)
        f += 1

    return fronts

# --------------------------------------
# Example function to build initial paths
# --------------------------------------
def build_population(start, end_nodes, population_size):
    """
    Creates 'population_size' random paths from start to any node in 'end_nodes'
    """
    total_population = []
    while len(total_population) < population_size:
        visited = set()
        path = [start]
        visited.add(start)

        while path[-1] not in end_nodes:
            curr = path[-1]
            unvisited_neighbors = [node for node in graph.get(curr, []) if node not in visited]
            if not unvisited_neighbors:
                break
            next_choice = choice(unvisited_neighbors)
            path.append(next_choice)
            visited.add(next_choice)

        if path[-1] in end_nodes and path not in total_population:
            total_population.append(path)

    return total_population

# --------------------------------
# Main Genetic Algorithm w/ Pareto
# --------------------------------
def genetic_algorithm_pareto_with_tracking(
    start,
    end_nodes,
    population_size=20,
    generations=50,
    crossover_rate=0.8,
    mutation_rate=0.2
):
    # Initialize: random population
    parent_population = build_population(start, end_nodes, population_size)

    # Initialize tracking lists
    best_latency_per_gen = []
    best_rate_per_gen = []
    avg_latency_per_gen = []
    avg_rate_per_gen = []

    for gen in range(generations):
        # Evaluate each path => compute cost = (rate, latency)
        costs = [cost_function(path) for path in parent_population]

        # Track metrics for this generation
        latencies = [cost[1] for cost in costs]
        rates = [cost[0] for cost in costs]
        best_latency_per_gen.append(min(latencies))  # Best latency
        best_rate_per_gen.append(max(rates))        # Best rate
        avg_latency_per_gen.append(sum(latencies) / len(latencies))  # Average latency
        avg_rate_per_gen.append(sum(rates) / len(rates))            # Average rate

        # Non-dominated sort => get list of fronts
        fronts = non_dominated_sort(costs)

        # Selection: pick from front 0, then front 1, etc. until population_size is reached
        new_population = []
        for front in fronts:
            if len(new_population) + len(front) <= population_size:
                new_population.extend([parent_population[i] for i in front])
            else:
                # Take subset if needed
                needed = population_size - len(new_population)
                selected_indices = random.sample(front, needed)
                new_population.extend([parent_population[i] for i in selected_indices])
            if len(new_population) >= population_size:
                break

        parent_population = new_population

        # Crossover
        offspring = []
        while len(offspring) < population_size * crossover_rate:
            parent1, parent2 = random.sample(parent_population, 2)
            common_nodes = set(parent1) & set(parent2) - {start}
            if common_nodes:
                crossover_node = random.choice(list(common_nodes))
                idx1 = parent1.index(crossover_node)
                idx2 = parent2.index(crossover_node)
                child1 = parent1[:idx1] + parent2[idx2:]
                child2 = parent2[:idx2] + parent1[idx1:]
                for child in (child1, child2):
                    if child[-1] in end_nodes:
                        offspring.append(child)

        # Mutation
        for child in offspring:
            if random.random() < mutation_rate and len(child) > 2:
                mutation_point = random.randint(1, len(child) - 2)
                subpath = build_population(child[mutation_point], end_nodes, 1)[0]
                child[mutation_point:] = subpath

        # Combine parents + offspring
        parent_population.extend(offspring)
        if len(parent_population) > population_size:
            costs = [cost_function(path) for path in parent_population]
            fronts = non_dominated_sort(costs)
            reduced_pop = []
            for front in fronts:
                if len(reduced_pop) + len(front) <= population_size:
                    reduced_pop.extend([parent_population[i] for i in front])
                else:
                    needed = population_size - len(reduced_pop)
                    selected_indices = random.sample(front, needed)
                    reduced_pop.extend([parent_population[i] for i in selected_indices])
                if len(reduced_pop) >= population_size:
                    break
            parent_population = reduced_pop

    # At the end, compute the final fronts again
    final_costs = [cost_function(path) for path in parent_population]
    final_fronts = non_dominated_sort(final_costs)

    # The “best” solutions (first front) are all Pareto-optimal
    best_front_indices = final_fronts[0]
    best_solutions = [parent_population[i] for i in best_front_indices]
    best_solutions_costs = [final_costs[i] for i in best_front_indices]

    return (
        best_solutions[0],
        best_solutions_costs[0][1],  # Best latency
        best_solutions_costs[0][0],  # Best rate
        best_latency_per_gen,
        best_rate_per_gen,
        avg_latency_per_gen,
        avg_rate_per_gen
    )


# -----------------
# Run the algorithm
# -----------------

# start_node = "Node_6"
# end_nodes  = ["Node_1", "Node_152"]  # Example end nodes
# population_size = 20
# generations = 50

# pareto_solutions, latency, rate = genetic_algorithm_pareto(
#     start_node,
#     end_nodes,
#     population_size,
#     generations
# )

